# 读取数据

In [1]:
from gspan_mining.config import parser
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from random import random
import sys

import numpy as np
import pandas as pd


import random 
from tqdm import tqdm

In [2]:
import os
os.getcwd()

'/Users/tangting/Project/gSpan'

# 1. 生成 Vertex 数据集

In [3]:
vertex_count = 1000
origin_ids = []
int_fetaures = []
float_fetaures = []
node_label_0 = []
node_label_1 = []
for i in range(vertex_count):
    origin_id = random.randint(1000000,100000000)
    if origin_id not in origin_ids:
        origin_ids.append(origin_id)
        int_fetaures.append([random.randint(10,100),random.randint(200,300)])
        float_fetaures.append([random.random()*10,random.random()*2000])
        node_label_0.append(int(random.random()*2))    
        node_label_1.append(int(random.random()*2))



# vertext_tb = pd.DataFrame(data=[origin_ids,int_fetaures,float_fetaures],columns=["origin_id","int_fetaures","float_fetaures"])
vertext_tb = pd.DataFrame({"origin_id" : origin_ids, \
                            "int_fetaures": int_fetaures, \
                            "float_fetaures" :float_fetaures,
                            "node_label_0":node_label_0,\
                           "node_label_1":node_label_1     })
vertext_tb["id"] = vertext_tb.index
vertext_tb.to_csv("data/vertex_data.csv",index=False,header=True)
vertext_tb


,origin_id,int_fetaures,float_fetaures,node_label_0,node_label_1,id
0,52913290,"[42, 208]","[0.2454909619968726, 1307.2011994037516]",1,1,0
1,93646093,"[39, 248]","[0.7541657659209611, 1363.0114087341558]",0,0,1
2,40314607,"[28, 228]","[0.5052281934221436, 1229.3731617196315]",0,1,2
3,24145916,"[56, 264]","[2.80854952588087, 1595.87503449946]",0,0,3
4,18161028,"[80, 202]","[8.264901190103746, 1701.0297639857856]",0,0,4
...,...,...,...,...,...,...
995,49322560,"[18, 270]","[0.19395678145262774, 1942.5509178158272]",1,1,995
996,13128935,"[86, 218]","[8.453353272818534, 1015.6500338982062]",0,1,996
997,17337074,"[18, 294]","[8.752197791817641, 1040.5840107607185]",1,0,997
998,86864896,"[74, 225]","[7.589070151722447, 1290.2239330452642]",1,0,998


# 2. 生成 Edge 数据集


In [4]:
vertex_count = len(origin_ids)
edge_count = vertex_count * 10
edge_src_ids = []
edge_dst_ids = []
edge_types = []
for i in range(edge_count):
    src_rand_id = random.randint(0,vertex_count-1)
    dst_rand_id = random.randint(0,vertex_count-1)
    if src_rand_id != dst_rand_id:
        src_id = origin_ids[src_rand_id]
        edge_src_ids.append(src_id)
        dst_id = origin_ids[dst_rand_id]
        edge_dst_ids.append(dst_id)
        edge_types.append(random.randint(0,5))

# vertext_tb = pd.DataFrame(data=[origin_ids,int_fetaures,float_fetaures],columns=["origin_id","int_fetaures","float_fetaures"])
edge_tb = pd.DataFrame({"src_origin_id" : edge_src_ids,"dst_origin_id": edge_dst_ids,"edge_type" :edge_types})

edge_tb.to_csv("data/edge_data.csv",index=False,header=True)


In [5]:

edge_tb_src = edge_tb.merge(vertext_tb[["id","origin_id"]], \
                            left_on="src_origin_id", \
                            right_on="origin_id", \
                            how="left") \
                                [["src_origin_id","dst_origin_id","edge_type","id"]].rename(columns = {"id":"src_id"})

edge_tb_dst = edge_tb_src.merge(vertext_tb[["id","origin_id"]], \
                            left_on="dst_origin_id", \
                            right_on="origin_id", \
                            how="left") \
                                [["src_origin_id","dst_origin_id","edge_type","src_id","id"]].rename(columns = {"id":"dst_id"})

edge_end = edge_tb_dst[["edge_type","src_id","dst_id"]]


edge_end
# edge_tb


,edge_type,src_id,dst_id
0,2,776,9
1,2,935,20
2,0,332,528
3,3,701,553
4,4,379,63
...,...,...,...
9988,2,292,376
9989,2,924,442
9990,1,354,355
9991,0,37,604


# 3.找到 origin_id => subgraph


In [6]:
# vertext_end, edge_end

vertext_end = vertext_tb[["id", "int_fetaures", "float_fetaures", "node_label_0", "node_label_1"]]
origin_ids = vertext_end[(vertext_end["node_label_0"]==0)&(vertext_end["node_label_1"])]
top1_neigh = pd.merge(origin_ids,edge_end,left_on="id",right_on="src_id").rename(columns={"id":"graph_id"})
top2_neigh = pd.merge(top1_neigh[["graph_id","dst_id"]].rename(columns={"dst_id":"id"}).drop_duplicates(),edge_end,left_on="id",right_on="src_id")
top3_neigh = pd.merge(top2_neigh[["graph_id","dst_id"]].rename(columns={"dst_id":"id"}).drop_duplicates(),edge_end,left_on="id",right_on="src_id")
top3_neigh.drop_duplicates()
# pd.merge(top1_neigh,edge_end,left_on="dst_id",right_on="src_id").rename(columns={"id":"graph_id"})
# origin_ids.join(edge_end,left)

,graph_id,id,edge_type,src_id,dst_id
0,2,373,2,373,294
1,2,373,5,373,248
2,2,373,0,373,538
3,2,373,4,373,29
4,2,373,4,373,197
...,...,...,...,...,...
220595,762,540,4,540,287
220596,762,540,4,540,350
220597,762,540,5,540,880
220598,762,540,3,540,718


In [7]:
graphs = pd.concat([top1_neigh[["graph_id","src_id","dst_id","edge_type"]],
            top2_neigh[["graph_id","src_id","dst_id","edge_type"]],
                top3_neigh[["graph_id","src_id","dst_id","edge_type"]]],axis=0).drop_duplicates()
origin_ids["graph_id"] = origin_ids["id"]
select_vertexes = pd.concat([graphs[["graph_id","src_id"]].rename(columns={"src_id":"id"}),
                    graphs[["graph_id","dst_id"]].rename(columns={"dst_id":"id"}),
                    origin_ids[["graph_id","id"]]],axis=0).drop_duplicates()


select_edges = graphs


/var/folders/23/8rnl11914v1_gmqkg83jtt8h0000gn/T/ipykernel_28273/2736574741.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  origin_ids["graph_id"] = origin_ids["id"]


## 过滤掉周围邻居没有黑节点的子图


In [8]:
select_vertexes = pd.merge(select_vertexes,vertext_end[["id","node_label_0","node_label_1"]],how="left")

In [9]:
# 修改图的编号

graph_id_map = select_vertexes[["graph_id"]].drop_duplicates().reset_index(drop=True)
graph_id_map["graph_new_id"] = graph_id_map.index

graph_id_map_end = graph_id_map.rename(columns={"graph_id":"origin_id"}).rename(columns={"graph_new_id":"graph_id"})
graph_id_map_end

,origin_id,graph_id
0,2,0
1,19,1
2,20,2
3,21,3
4,31,4
...,...,...
229,988,229
230,989,230
231,990,231
232,994,232


In [10]:
select_vertexes = pd.merge(select_vertexes,graph_id_map,on="graph_id").drop(columns=["graph_id"]).rename(columns = {"graph_new_id":"graph_id"})
select_vertexes

,id,node_label_0,node_label_1,graph_id
0,2,0,1,0
1,491,1,1,0
2,102,1,1,0
3,285,0,0,0
4,441,0,1,0
...,...,...,...,...
146582,16,0,0,233
146583,951,0,0,233
146584,366,0,1,233
146585,362,0,0,233


In [11]:

select_vertexes.to_csv("graphdata/vertexes.csv",index=False,header=True)

edge_end.to_csv("graphdata/edges.csv",index=False,header=True)
graph_id_map_end.to_csv("graphdata/graph_map.csv",index=False,header=True)

# 4. 子图挖掘

In [2]:
from gspan_mining.config import parser
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from random import random
import sys

from ceb_gspan_mining.config import parser
from ceb_gspan_mining.ceb_gspan import gSpan

import numpy as np
import pandas as pd


import random 
def main(FLAGS=None):
    """Run gSpan."""

    if FLAGS is None:
        FLAGS, _ = parser.parse_known_args(args=["graphdata/graph.data.5"])

    if not os.path.exists(FLAGS.database_file_name):
        print('{} does not exist.'.format(FLAGS.database_file_name))
        sys.exit()

    graph_dict = {"group_id":"graph_id","vertex_id":"id","edge_src":"src_id","edge_dst":"dst_id","origin_id":"origin_id"}
    # graph_dict =  {"group_id":"graph_id","vertex_id":"id","edge_src":"src","edge_dst":"dst"}
    gs = gSpan(
        database_file_name=FLAGS.database_file_name,
        min_support=FLAGS.min_support,
        min_num_vertices=FLAGS.lower_bound_of_num_vertices,
        max_num_vertices=FLAGS.upper_bound_of_num_vertices,
        max_ngraphs=FLAGS.num_graphs,
        is_undirected=(not FLAGS.directed),
        verbose=FLAGS.verbose,
        visualize=FLAGS.plot,
        where=FLAGS.where,
        check_featureblack = FLAGS.check_featureblack,
        graph_dict = graph_dict
    )

    gs.run()
    gs.time_stats()

    
    return gs


if __name__ == '__main__':
    args_str = '--min_support 100 --directed True --lower_bound_of_num_vertices 4 --plot True --where True --check_featureblack True graphdata/'
    FLAGS, _ = parser.parse_known_args(args=args_str.split())
    main(FLAGS)


100%|██████████| 234/234 [00:06<00:00, 38.83it/s]


1

# Other

In [15]:
graph_id = graph_dict['group_id']
vertex_id = graph_dict['vertex_id']
edge_src = graph_dict['edge_src']
edge_dst = graph_dict['edge_dst']

edges = pd.read_csv(database_file_name+"/edges.csv", sep=',')
vertexes = pd.read_csv(database_file_name+"/vertexes.csv", sep=',')
vertexes.fillna(0)
vertexes.rename(columns={graph_id: 'group_id', vertex_id: "vertex_id"}, inplace=True)

vertexes["node_label_change"] = vertexes["node_label_1"] * 2+vertexes["node_label_0"]

edges.rename(columns={edge_src: 'edge_src',
                edge_dst: "edge_dst"}, inplace=True)

graphs = vertexes["group_id"].drop_duplicates()
graphs_len = len(graphs)
for i in tqdm(range(graphs_len)):
    g_id = graphs.iloc[i]
    vertexes_pd = vertexes[vertexes.group_id == g_id]
    mask = edges[edges.edge_src.isin(vertexes_pd["vertex_id"])
                        & edges.edge_dst.isin(vertexes_pd["vertex_id"])].index
    edges.loc[mask,"group_id"] =  g_id
    edges.loc[mask,"e_id"] =  [i for i in range(len(mask))]

NameError: name 'graph_dict' is not defined

vertexes

In [127]:
# vertexes
# edges
edges_src = pd.merge(edges,vertexes[["group_id","vertex_id","node_label_change"]],how="left",left_on="edge_src",right_on="vertex_id")

# .merge(vertexes,how="left",left_on="edge_dst",right_on="vertex_id") 
edges_src.rename(columns={"group_id":"group_id_a","vertex_id":"vertex_id_a","node_label_change":"node_label_change_a"},inplace=True)
edges_src = edges_src[edges_src.group_id_x==edges_src.group_id_y]

edges_dst = edges_src.merge(vertexes[["group_id","vertex_id","node_label_change"]],how="left",left_on="edge_dst",right_on="vertex_id")
edges_dst.rename(columns={"group_id":"group_id_b","vertex_id":"vertex_id_b","node_label_change":"node_label_change_b"},inplace=True)
edges_dst = edges_dst[edges_dst.group_id_x==edges_dst.group_id_b]

edges_final = edges_dst[["edge_src", 'edge_dst', 'edge_type', 'group_id_x', 'e_id',
        'node_label_change_a',  'node_label_change_b']]

In [137]:

vlb_counter = vertexes.groupby("node_label_change")["vertex_id"]
vlb_counted = vertexes.groupby(["node_label_change","group_id"])["vertex_id"]

In [138]:
vevlb_counter = edges_final.groupby(["node_label_change_a","edge_type","node_label_change_b"])["e_id"]
vevlb_counted = edges_final.groupby(["group_id_x","node_label_change_a","edge_type","node_label_change_b"])["e_id"]

In [146]:
b = vevlb_counter.count().reset_index()
# b

get_forward_root_edges result

In [147]:
get_forward_first = edges_final[edges_final.edge_src<edges_final.edge_dst]


In [148]:
# vevlb:(v.vlb, e.elb, g.vertices[e.to].vlb) , 
# projected:PDFS(gid, e, None)

get_forward_first_start = get_forward_first[get_forward_first.node_label_change_a==1]
get_forward_first_start["frm"] = 0
get_forward_first_start["to"] = 1



,edge_src,edge_dst,edge_type,group_id_x,e_id,node_label_change_a,node_label_change_b
1,22,26,3,20.0,0.0,3,1
3,11,18,1,10.0,0.0,0,1
6,23,30,5,0.0,3.0,1,3
10,9,12,4,0.0,5.0,1,0
13,23,24,2,20.0,2.0,2,3
...,...,...,...,...,...,...,...
84066,4925,4934,0,490.0,2968.0,3,2
84067,498,4946,1,490.0,2969.0,3,2
84070,4912,4934,1,490.0,2972.0,3,2
84071,498,4935,3,490.0,2973.0,3,3


In [63]:
edges

,Unnamed: 0,edge_src,edge_dst,edge_type,group_id,e_id
0,0,112,134,4,0,NaN
1,1,158,121,3,0,NaN
2,2,150,148,2,0,NaN
3,3,117,13,3,0,NaN
4,4,125,150,2,0,NaN
...,...,...,...,...,...,...
57634,57634,4934,4937,0,64,64.0
57635,57635,4923,498,0,65,65.0
57636,57636,4916,495,1,66,66.0
57637,57637,497,4932,2,67,67.0
